### Projeto com Feedback 4

#### Prevendo Customer Churn em Operadoras de Telecom

Identificar Rotatividade dos clientes (customer churn) de operadora de TELECOM. Prever se os clientes abandonarão o relacionamento comercial. Análise é um trabalho de classificação (sim/não)

Projeto utilizando Spark

In [1]:
#imports

from pyspark.sql import Row
from pyspark.ml.feature import StringIndexer
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LinearSVC
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

criando a sessão do spark, definindo o master como local, definindo nome como sparkML

In [2]:
spSession = SparkSession.builder.master('local').appName('sparkMl').getOrCreate()

criando um RDD a partir do arquivo importado usando spark context

In [3]:
telecomRDD = sc.textFile('projeto4_telecom_treino.csv')

fazendo split dos dados atravéz do seu separador. criando novo RDD (RDD são imutáveis), Removendo a primeira coluna

In [4]:
telecomRDD2 = telecomRDD.map(lambda x: x.split(',')[1:])

In [5]:
telecomRDD2.take(1)

[['"state"',
  '"account_length"',
  '"area_code"',
  '"international_plan"',
  '"voice_mail_plan"',
  '"number_vmail_messages"',
  '"total_day_minutes"',
  '"total_day_calls"',
  '"total_day_charge"',
  '"total_eve_minutes"',
  '"total_eve_calls"',
  '"total_eve_charge"',
  '"total_night_minutes"',
  '"total_night_calls"',
  '"total_night_charge"',
  '"total_intl_minutes"',
  '"total_intl_calls"',
  '"total_intl_charge"',
  '"number_customer_service_calls"',
  '"churn"']]

### convertendo rdd para dataframe do spark

In [6]:
# removendo primeira linha (header)
telecomRDD3 = telecomRDD2.filter(lambda x: "\"state\"" not in x)

In [7]:
telecomRDD4 = telecomRDD3.map(lambda x: Row(ACCOUNT_LENGHT=float(x[1]), INTERNATIONAL_PLAN=x[3],
                                            VOICE_MAIL_PLAN=x[4], NUMBER_VMAIL_MESSAGES=float(x[5]), 
                                            TOTAL_DAY_MINUTES=float(x[6]), TOTAL_DAY_CALLS=float(x[7]),
                                            TOTAL_DAY_CHARGE=float(x[8]),TOTAL_EVE_MINUTES=float(x[9]),
                                            TOTAL_EVE_CALLS=float(x[10]),TOTAL_EVE_CHARGE=float(x[11]),
                                            TOTAL_NIGHT_MINUTES=float(x[12]), TOTAL_NIGHT_CALLS=float(x[13]),
                                            TOTAL_NIGHT_CHARGE=float(x[14]), TOTAL_INTL_MINUTES=float(x[15]),
                                            TOTAL_INTL_CALLS=float(x[16]), TOTAL_INTL_CHARGE=float(x[17]),
                                            NUMBER_CUSTOMER_SERVICE_CALLS=float(x[18]), CHURN=x[19]))
telecomRDD4.take(5)

[Row(ACCOUNT_LENGHT=128.0, INTERNATIONAL_PLAN='"no"', VOICE_MAIL_PLAN='"yes"', NUMBER_VMAIL_MESSAGES=25.0, TOTAL_DAY_MINUTES=265.1, TOTAL_DAY_CALLS=110.0, TOTAL_DAY_CHARGE=45.07, TOTAL_EVE_MINUTES=197.4, TOTAL_EVE_CALLS=99.0, TOTAL_EVE_CHARGE=16.78, TOTAL_NIGHT_MINUTES=244.7, TOTAL_NIGHT_CALLS=91.0, TOTAL_NIGHT_CHARGE=11.01, TOTAL_INTL_MINUTES=10.0, TOTAL_INTL_CALLS=3.0, TOTAL_INTL_CHARGE=2.7, NUMBER_CUSTOMER_SERVICE_CALLS=1.0, CHURN='"no"'),
 Row(ACCOUNT_LENGHT=107.0, INTERNATIONAL_PLAN='"no"', VOICE_MAIL_PLAN='"yes"', NUMBER_VMAIL_MESSAGES=26.0, TOTAL_DAY_MINUTES=161.6, TOTAL_DAY_CALLS=123.0, TOTAL_DAY_CHARGE=27.47, TOTAL_EVE_MINUTES=195.5, TOTAL_EVE_CALLS=103.0, TOTAL_EVE_CHARGE=16.62, TOTAL_NIGHT_MINUTES=254.4, TOTAL_NIGHT_CALLS=103.0, TOTAL_NIGHT_CHARGE=11.45, TOTAL_INTL_MINUTES=13.7, TOTAL_INTL_CALLS=3.0, TOTAL_INTL_CHARGE=3.7, NUMBER_CUSTOMER_SERVICE_CALLS=1.0, CHURN='"no"'),
 Row(ACCOUNT_LENGHT=137.0, INTERNATIONAL_PLAN='"no"', VOICE_MAIL_PLAN='"no"', NUMBER_VMAIL_MESSAGES=0.0,

In [8]:
telecomDF = spSession.createDataFrame(telecomRDD4)

#### Convertendo variavel categórica em numérica

In [9]:
stringIndice = StringIndexer(inputCol='INTERNATIONAL_PLAN', outputCol='IDX_INTERNATIONAL_PLAN')
si_model = stringIndice.fit(telecomDF)
telecomDF = si_model.transform(telecomDF)

In [10]:
telecomDF.head()

Row(ACCOUNT_LENGHT=128.0, INTERNATIONAL_PLAN='"no"', VOICE_MAIL_PLAN='"yes"', NUMBER_VMAIL_MESSAGES=25.0, TOTAL_DAY_MINUTES=265.1, TOTAL_DAY_CALLS=110.0, TOTAL_DAY_CHARGE=45.07, TOTAL_EVE_MINUTES=197.4, TOTAL_EVE_CALLS=99.0, TOTAL_EVE_CHARGE=16.78, TOTAL_NIGHT_MINUTES=244.7, TOTAL_NIGHT_CALLS=91.0, TOTAL_NIGHT_CHARGE=11.01, TOTAL_INTL_MINUTES=10.0, TOTAL_INTL_CALLS=3.0, TOTAL_INTL_CHARGE=2.7, NUMBER_CUSTOMER_SERVICE_CALLS=1.0, CHURN='"no"', IDX_INTERNATIONAL_PLAN=0.0)

In [11]:
stringIndice = StringIndexer(inputCol='VOICE_MAIL_PLAN', outputCol='IDX_VOICE_MAIL_PLAN')
si_model = stringIndice.fit(telecomDF)
telecomDF = si_model.transform(telecomDF)

In [12]:
telecomDF.head()

Row(ACCOUNT_LENGHT=128.0, INTERNATIONAL_PLAN='"no"', VOICE_MAIL_PLAN='"yes"', NUMBER_VMAIL_MESSAGES=25.0, TOTAL_DAY_MINUTES=265.1, TOTAL_DAY_CALLS=110.0, TOTAL_DAY_CHARGE=45.07, TOTAL_EVE_MINUTES=197.4, TOTAL_EVE_CALLS=99.0, TOTAL_EVE_CHARGE=16.78, TOTAL_NIGHT_MINUTES=244.7, TOTAL_NIGHT_CALLS=91.0, TOTAL_NIGHT_CHARGE=11.01, TOTAL_INTL_MINUTES=10.0, TOTAL_INTL_CALLS=3.0, TOTAL_INTL_CHARGE=2.7, NUMBER_CUSTOMER_SERVICE_CALLS=1.0, CHURN='"no"', IDX_INTERNATIONAL_PLAN=0.0, IDX_VOICE_MAIL_PLAN=1.0)

In [13]:
stringIndice = StringIndexer(inputCol='CHURN', outputCol='IDX_CHURN')
si_model = stringIndice.fit(telecomDF)
telecomDF = si_model.transform(telecomDF)

In [14]:
telecomDF.head()

Row(ACCOUNT_LENGHT=128.0, INTERNATIONAL_PLAN='"no"', VOICE_MAIL_PLAN='"yes"', NUMBER_VMAIL_MESSAGES=25.0, TOTAL_DAY_MINUTES=265.1, TOTAL_DAY_CALLS=110.0, TOTAL_DAY_CHARGE=45.07, TOTAL_EVE_MINUTES=197.4, TOTAL_EVE_CALLS=99.0, TOTAL_EVE_CHARGE=16.78, TOTAL_NIGHT_MINUTES=244.7, TOTAL_NIGHT_CALLS=91.0, TOTAL_NIGHT_CHARGE=11.01, TOTAL_INTL_MINUTES=10.0, TOTAL_INTL_CALLS=3.0, TOTAL_INTL_CHARGE=2.7, NUMBER_CUSTOMER_SERVICE_CALLS=1.0, CHURN='"no"', IDX_INTERNATIONAL_PLAN=0.0, IDX_VOICE_MAIL_PLAN=1.0, IDX_CHURN=0.0)

Removendo colunas que não serão utilizadas

In [15]:
columns_to_drop = ['INTERNATIONAL_PLAN', 'VOICE_MAIL_PLAN','CHURN']
telecomDF = telecomDF.drop(*columns_to_drop)

In [16]:
telecomDF.head()

Row(ACCOUNT_LENGHT=128.0, NUMBER_VMAIL_MESSAGES=25.0, TOTAL_DAY_MINUTES=265.1, TOTAL_DAY_CALLS=110.0, TOTAL_DAY_CHARGE=45.07, TOTAL_EVE_MINUTES=197.4, TOTAL_EVE_CALLS=99.0, TOTAL_EVE_CHARGE=16.78, TOTAL_NIGHT_MINUTES=244.7, TOTAL_NIGHT_CALLS=91.0, TOTAL_NIGHT_CHARGE=11.01, TOTAL_INTL_MINUTES=10.0, TOTAL_INTL_CALLS=3.0, TOTAL_INTL_CHARGE=2.7, NUMBER_CUSTOMER_SERVICE_CALLS=1.0, IDX_INTERNATIONAL_PLAN=0.0, IDX_VOICE_MAIL_PLAN=1.0, IDX_CHURN=0.0)

### Verificando estatistica descritiva dos dados

In [17]:
telecomDF.describe('ACCOUNT_LENGHT','NUMBER_VMAIL_MESSAGES','TOTAL_DAY_MINUTES','TOTAL_DAY_CALLS','TOTAL_DAY_CHARGE').show()

+-------+------------------+---------------------+------------------+------------------+------------------+
|summary|    ACCOUNT_LENGHT|NUMBER_VMAIL_MESSAGES| TOTAL_DAY_MINUTES|   TOTAL_DAY_CALLS|  TOTAL_DAY_CHARGE|
+-------+------------------+---------------------+------------------+------------------+------------------+
|  count|              3333|                 3333|              3333|              3333|              3333|
|   mean|101.06480648064806|    8.099009900990099|179.77509750975088|100.43564356435644|30.562307230723054|
| stddev| 39.82210592859562|   13.688365372038595|  54.4673892023714|20.069084207300907| 9.259434553930506|
|    min|               1.0|                  0.0|               0.0|               0.0|               0.0|
|    max|             243.0|                 51.0|             350.8|             165.0|             59.64|
+-------+------------------+---------------------+------------------+------------------+------------------+



In [18]:
telecomDF.describe('TOTAL_EVE_MINUTES','TOTAL_EVE_CALLS','TOTAL_EVE_CHARGE','TOTAL_NIGHT_MINUTES','TOTAL_NIGHT_CALLS').show()

+-------+-----------------+------------------+------------------+-------------------+------------------+
|summary|TOTAL_EVE_MINUTES|   TOTAL_EVE_CALLS|  TOTAL_EVE_CHARGE|TOTAL_NIGHT_MINUTES| TOTAL_NIGHT_CALLS|
+-------+-----------------+------------------+------------------+-------------------+------------------+
|  count|             3333|              3333|              3333|               3333|              3333|
|   mean|200.9803480348034|100.11431143114311|17.083540354035424| 200.87203720372082|100.10771077107711|
| stddev|  50.713844425812|19.922625293943103|4.3106676431103415|  50.57384701365836|19.568609346058555|
|    min|              0.0|               0.0|               0.0|               23.2|              33.0|
|    max|            363.7|             170.0|             30.91|              395.0|             175.0|
+-------+-----------------+------------------+------------------+-------------------+------------------+



In [19]:
telecomDF.describe('TOTAL_NIGHT_CHARGE','TOTAL_INTL_MINUTES','TOTAL_INTL_CALLS','TOTAL_INTL_CHARGE','NUMBER_CUSTOMER_SERVICE_CALLS').show()

+-------+------------------+------------------+------------------+------------------+-----------------------------+
|summary|TOTAL_NIGHT_CHARGE|TOTAL_INTL_MINUTES|  TOTAL_INTL_CALLS| TOTAL_INTL_CHARGE|NUMBER_CUSTOMER_SERVICE_CALLS|
+-------+------------------+------------------+------------------+------------------+-----------------------------+
|  count|              3333|              3333|              3333|              3333|                         3333|
|   mean|  9.03932493249324|10.237293729372928|4.4794479447944795| 2.764581458145822|           1.5628562856285628|
| stddev| 2.275872837660028|2.7918395484084138|2.4612142705460935|0.7537726126630463|           1.3154910448664763|
|    min|              1.04|               0.0|               0.0|               0.0|                          0.0|
|    max|             17.77|              20.0|              20.0|               5.4|                          9.0|
+-------+------------------+------------------+------------------+------

### Verificando a correlação entre as variáveis e o target

In [20]:
# encontrando a correlação entre a variáveis
for i in telecomDF.columns:
    if not(isinstance(telecomDF.select(i).take(1)[0][0], str)):
        print("Correlação entre a variável target IDX_CHURN e a variavel ", i, telecomDF.stat.corr('IDX_CHURN',i))

Correlação entre a variável target IDX_CHURN e a variavel  ACCOUNT_LENGHT 0.016540742243674286
Correlação entre a variável target IDX_CHURN e a variavel  NUMBER_VMAIL_MESSAGES -0.08972796983506418
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_DAY_MINUTES 0.2051508292613899
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_DAY_CALLS 0.018459311608577066
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_DAY_CHARGE 0.20515074317015397
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_EVE_MINUTES 0.09279579031259168
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_EVE_CALLS 0.009233131913077921
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_EVE_CHARGE 0.09278603942871391
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_NIGHT_MINUTES 0.03549285342127406
Correlação entre a variável target IDX_CHURN e a variavel  TOTAL_NIGHT_CALLS 0.006141203007399843
Correlação entre a variável t

In [21]:
# mutando o tipo da variavel target para integer (tipo foi alterado durante o pré-processamento)
telecomDF = telecomDF.withColumn("IDX_CHURN",telecomDF.IDX_CHURN.cast(IntegerType()))

### Criando um vetor denso

In [22]:
# criando vetor denso
def transformDenseVector(row):
    obj = (row["IDX_CHURN"], Vectors.dense([row["ACCOUNT_LENGHT"], 
                                          row["NUMBER_VMAIL_MESSAGES"], 
                                          row["TOTAL_DAY_MINUTES"],
                                           row["TOTAL_DAY_CALLS"],
                                           row["TOTAL_DAY_CHARGE"],
                                           row["TOTAL_EVE_MINUTES"],
                                           row["TOTAL_EVE_CALLS"],
                                           row["TOTAL_EVE_CHARGE"],
                                           row["TOTAL_NIGHT_MINUTES"],
                                           row["TOTAL_NIGHT_CALLS"],
                                           row["TOTAL_NIGHT_CHARGE"],
                                           row["TOTAL_INTL_MINUTES"],
                                           row["TOTAL_INTL_CALLS"],
                                           row["TOTAL_INTL_CHARGE"],
                                           row["NUMBER_CUSTOMER_SERVICE_CALLS"],
                                           row["IDX_INTERNATIONAL_PLAN"],
                                           row["IDX_VOICE_MAIL_PLAN"]]))
    return obj

In [23]:
telecomDF.take(1)

[Row(ACCOUNT_LENGHT=128.0, NUMBER_VMAIL_MESSAGES=25.0, TOTAL_DAY_MINUTES=265.1, TOTAL_DAY_CALLS=110.0, TOTAL_DAY_CHARGE=45.07, TOTAL_EVE_MINUTES=197.4, TOTAL_EVE_CALLS=99.0, TOTAL_EVE_CHARGE=16.78, TOTAL_NIGHT_MINUTES=244.7, TOTAL_NIGHT_CALLS=91.0, TOTAL_NIGHT_CHARGE=11.01, TOTAL_INTL_MINUTES=10.0, TOTAL_INTL_CALLS=3.0, TOTAL_INTL_CHARGE=2.7, NUMBER_CUSTOMER_SERVICE_CALLS=1.0, IDX_INTERNATIONAL_PLAN=0.0, IDX_VOICE_MAIL_PLAN=1.0, IDX_CHURN=0)]

In [24]:
# converte o dataframe para RDD, para usar os recursos do SPARK, e com a função map aplica a função transformVar, criando um vetor denso
telecomRDD4 = telecomDF.rdd.map(transformDenseVector)

# convert novamente para dataframe do spark
telecomDF = spSession.createDataFrame(telecomRDD4,["label", "features"])

# exibe as primeiras 10 linhas
telecomDF.select("label","features").show(10)

+-----+--------------------+
|label|            features|
+-----+--------------------+
|    0|[128.0,25.0,265.1...|
|    0|[107.0,26.0,161.6...|
|    0|[137.0,0.0,243.4,...|
|    0|[84.0,0.0,299.4,7...|
|    0|[75.0,0.0,166.7,1...|
|    0|[118.0,0.0,223.4,...|
|    0|[121.0,24.0,218.2...|
|    0|[147.0,0.0,157.0,...|
|    0|[117.0,0.0,184.5,...|
|    0|[141.0,37.0,258.6...|
+-----+--------------------+
only showing top 10 rows



### Utilizando Linear suport vector machine para classificação

In [25]:
# Criando o modelo de classificação Linear suport vector machine
model = LinearSVC(maxIter=10, regParam=0.1)
modelo = model.fit(telecomDF)

In [26]:
# Previsões
previsoes = modelo.transform(telecomDF)
previsoes.show()

+-----+--------------------+--------------------+----------+
|label|            features|       rawPrediction|prediction|
+-----+--------------------+--------------------+----------+
|    0|[128.0,25.0,265.1...|[1.14907605389987...|       0.0|
|    0|[107.0,26.0,161.6...|[1.22238680462497...|       0.0|
|    0|[137.0,0.0,243.4,...|[1.09743462342406...|       0.0|
|    0|[84.0,0.0,299.4,7...|[0.77334269223962...|       0.0|
|    0|[75.0,0.0,166.7,1...|[0.99062449534574...|       0.0|
|    0|[118.0,0.0,223.4,...|[1.00768460489193...|       0.0|
|    0|[121.0,24.0,218.2...|[1.21527890248316...|       0.0|
|    0|[147.0,0.0,157.0,...|[0.86063826522282...|       0.0|
|    0|[117.0,0.0,184.5,...|[1.09033678541659...|       0.0|
|    0|[141.0,37.0,258.6...|[1.17142894534126...|       0.0|
|    1|[65.0,0.0,129.1,1...|[1.13064242261646...|       0.0|
|    0|[74.0,0.0,187.7,1...|[1.14638914677247...|       0.0|
|    0|[168.0,0.0,128.8,...|[0.96281719610780...|       0.0|
|    0|[95.0,0.0,156.6,8

In [27]:
# Avaliando a acurácia
previsoes = previsoes.withColumn("prediction",previsoes.prediction.cast(DoubleType()))
evaluator = MulticlassClassificationEvaluator(predictionCol = "prediction", labelCol = "label", metricName = "accuracy")
evaluator.evaluate(previsoes) 

0.8550855085508551